In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

In [2]:
#### load class
train_label = pd.read_csv('temp_data/train_label_lite.csv')
# hasher = pd.read_csv('test_id.csv')
label_map = {'retained':0,'2month':1,'month':2,'week':3}
y_train = pd.Series([label_map[l] for l in train_label.label])
inv_map = {label_map[k]:k for k in label_map.keys()}

In [3]:
X_train = pd.read_csv('X_train_final.csv')

In [4]:
train_data=lgb.Dataset(X_train, label=np.array(y_train).reshape(-1))

In [ ]:
param = {'objective':'softmax'}
#http://lightgbm.readthedocs.io/en/latest/Parameters.html
#core parameters
param['boosting'] = 'dart'
param['num_iterations'] = 500 ### cv로 적절한 값 찾기
param['num_leaves']= 1000
param['metric'] = 'multi_error'
param['n_jobs']=-1
param['learning_rate']=0.1
param['treelearner']='serial'
#gpu 설정하려면 좀 더 봐야 함
param['device_type']='gpu'
param['seed']=0

#learning control parameters
param['max_depth']=100
param['min_data_in_leaf']=200
#param['lambda_l1']
#param['lambda_l2']
#param['drop_rate']
#param['max_drop']
#param['top_k']

#objectiv parameters
param['num_class']=4
param['is_unbalance']=False

In [ ]:
lgb.train(param)

In [ ]:
#### cross validation(for NA)
kfold = StratifiedKFold(n_splits = 5 ,random_state = 7,shuffle=True).split(X_train, y_train)
scores = []
predict_set = []
for k, (train, test) in enumerate(kfold):
    bst = lgb.train(param,train_data)
    score = model.score(X_train.iloc[test,:], y_train[test])
    scores.append(score)
    print('Fold: %s, Class dist.: %s, F1: %.3f' % (k+1,np.bincount(y_train[train]), score))
    ### predict
    y_pred = model.predict(X_train.iloc[test,:])
    predict_set += [(x,inv_map[y_pred[i]]) for i,x in enumerate(test)]
print('\nCV F1: %.6f +/- %.6f' % (np.mean(scores), np.std(scores)))

In [ ]:
#모델 object를 bst에 저장
#early_stopping_roinds - model의 성능이 parameter로 준 round동안 개선되지 않으면 stop함
evals_result = {} 
bst = lgb.train(param, train_data, num_round, valid_sets=[val_data],early_stopping_rounds=4,evals_result=evals_result)

In [ ]:
#crossvalidation
num_round = 100
lgb.cv(param, train_data, num_round, nfold=5, early)